# Analog input

Slow analog inputs provide a convenient way to convenient way to acquire slow changing analog signals.
All four analog inputs are located on the E2 connector, depicted on the picture below and have a maximum input voltage rating of 7 V .

![alt text](img/Extension_connector.png "STEMlab extention connector.")

## Overlay

We will start with loading the FPGA bitstream and the device tree overlay.

In [ ]:
from redpitaya.overlay.mercury import mercury as FPGA
overlay = FPGA()

The next line loads the **analog_in** module that allows python to read  **Analog inputs** on the STEAMlab board.

We will start with one analog input, lets select **Analog input 0**.

In [ ]:
singleInput = FPGA.analog_in(0)

Here we will read the value on the input and print the value.

In [ ]:
print(singleInput.read())

The value printed above is the voltage applied on the  **Analog input 0** pin measured in Volts.

Lets make a formatted output where we will make that apparent. Also we will format the output, so that only 2 decimal places will be displayed.

In [ ]:
print('Measured voltage on Analog input pin 0 is {:.2f} V'.format(singleInput.read()))

## Reading all 4 Analog inputs

Here we use a for loop to declare all 4 channels.

Then we will use another for loop to read the values from all 4 channels. We will use the same formatting from the previous example.

In [ ]:
chn = 4
channels = [FPGA.analog_in(ch) for ch in range(chn)]

for ch in range(chn):
    print('{:.2f} V'.format(channels[ch].read()))

## Logging and plot live of data

In this example we will take a measurement on all 4 analog inputs every 0.5 s. Measured points will be stored into a list `y` and visualized with BokehJS.

### Generate data

To generate data we will use jupyters ability to run more than one notebook simultaneously. However before we do that we will have to manually connect output pins to the input pins as depicted in the picture bellow.

![analog Loop Back](img/analogLoopBack.png "STEMlab slow analog loop back.")


Now that we have created a phisical loop back open the [analog signal generator](analog_signal_generator.ipynb)  notebook run all the cells and return to this notebook. This will start an infinite loop that will generate sinus signals on all four analog outputs with different amplitudes.

**Note:**  
We do not recomend running more than 4 notebooks at the same time due to limited resources on STEMlab.

Since we want to make measurements every 0.5 second we will need to import time module and to be able plot data we will import a few modules from bokeh.

In [ ]:
import time

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure
from bokeh.resources import INLINE 
output_notebook(resources=INLINE)

Next we will prepare the parameters of the plot, such as line colors, axis labels, etc.

We will also prepare an empty plot that will be populated from the next cell.

In [ ]:
colors = ('red', 'blue', 'green', 'orange')
hover = HoverTool(mode = 'vline', tooltips=[("T", "@x"), ("V", "@y")])
tools = "pan,wheel_zoom,box_zoom,reset,crosshair"
p = figure(plot_height=400, plot_width=900, title="XADC log", toolbar_location="above", tools=(tools, hover))
p.xaxis.axis_label='time [s]'
p.y_range=Range1d(0, 2)
p.yaxis.axis_label='voltage [V]'

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

Here we will start sampling Analog inputs while also populating the plot. While the sampling rate is set to 0.5 sample per second, the actual sampling rate is lower as we use sleep function to delay the next sampling by 0.5 second, however we update the the plot in the same loop, which also takes time, since the sampling rate is lower.

In [ ]:
T = 0.01 # sampling period
x=[]
y=[ [], [], [], [] ]

r = [p.line(x, y[ch], line_width=1, line_alpha=0.7, color=colors[ch], legend="AI "+str(ch)) for ch in range(chn)]

# take 60 samples while simultaneously plotting the plot
for i in range(60):
    x.append(0) if i == 0 else x.append(x[-1]+T)
    for ch in range(chn):
        r[ch].data_source.data['y'].append(channels[ch].read())
    
    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=target)
    time.sleep(T)